# Data loader

In [158]:
import pandas as pd

from sklearn.model_selection import RepeatedStratifiedKFold
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1410)

class DatasetLoader:
    def get(self, name):
        print(f"loading {name}")
        if name == "fnkdd":
            data = pd.read_csv("/home/rkozik/Desktop/swarog_exp_disk/datasets/fakenewskdd/train.csv",sep="\t")
            data.head()
            body = data["text"].values
            labels = 1-data["label"].values
            total_number_of_claims = data.shape[0]
            print("total_number_of_claims=",total_number_of_claims)
            print("labels fake=",sum(labels),"real=", len(labels)-sum(labels))
        
        if name == "mmcovid":
            data = pd.read_csv("/media/rkozik/02FF-A831/data/swarog/datasets/mmcovid/news_collection.csv",sep="\t")
            data["label"] = [ 1 if v =="fake" else 0 for v in data["label"]]
            data["text"] = [ str(v) for v in data["text"]]
            data = data[data["lang"] == "en"]

            body = data["text"].values
            labels = data["label"].values
            total_number_of_claims = data.shape[0]
            print("total_number_of_claims=",total_number_of_claims)
            print("labels fake=",sum(labels),"real=", len(labels)-sum(labels))
        
        if name == "liar":
            data = pd.read_csv("/media/rkozik/02FF-A831/data/swarog/datasets/liar.csv", sep="\t",encoding="utf-8")
            def mpx(x):
                if x in [0,2]:
                    return 0
                elif x in [4,5]:
                    return 1
                else:
                    return -1
            data["text"] = data["statement"]
            data["label"] = [mpx(x) for x in data["label"]]
            data=data[ data["label"] != -1] 
            body = data["text"].values
            labels = data["label"].values
            total_number_of_claims = data.shape[0]
            print("total_number_of_claims=",total_number_of_claims)
            print("labels fake=",sum(labels),"real=", len(labels)-sum(labels))

        if name == "covidfn":
            data = pd.read_csv("covid_fake_news.csv", sep=",")
            body = data["headlines"].values
            labels = 1 - data["outcome"].values
            total_number_of_claims = data.shape[0]
            print("total_number_of_claims=",total_number_of_claims)
            print("labels fake=",sum(labels),"real=", len(labels)-sum(labels))
        
        return body, labels, total_number_of_claims

# Utility functions

In [159]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output



class Metrics:
    def __init__(self):
        self.scores = {
            'Accuracy': {'func': accuracy_score},
            'Balanced Accuracy': {'func': balanced_accuracy_score},
            'F1': {'func': f1_score},
            'Precision': {'func': precision_score},
            'Recall': {'func': recall_score},
            'G-mean': {'func': geometric_mean_score}
        }
        
        for score_name, score_dict in self.scores.items():
            score_dict["list"] = []
            score_dict["lab"] = []

    def update(self, actual, prediction):
        for score_name, score_dict in self.scores.items():
            if score_name in ["F1","Precision","Recall","G-mean"]:
                scorvaln = score_dict['func'](actual, prediction, average=None)
                score_dict['lab'].append(scorvaln)
                scorval = score_dict['func'](actual, prediction, average="weighted")
                score_dict['list'].append(scorval)
                #print(score_name, scorval, scorvaln)  
            else:
                scorval=score_dict['func'](actual, prediction)
                score_dict['list'].append(scorval)
                
    def print_table(self, labels=None):
        # Print stats
        scores = self.scores
        numlabels = scores["F1"]["lab"][0].shape[0]
        scores["F1"]["lab"][0].shape[0] 
        head = "  %-20s  %-10s  " +  numlabels * " %-10s  " 
        headv = ["Score", "Average"]
        if labels:
            headv.extend([labels[i] for i in range(numlabels)])
        else:
            headv.extend(["Lab:"+str(i+1) for i in range(numlabels)])
        row=head % tuple(headv)
        # table header
        print("―"*len(row))
        print(row)
        print("―"*len(row))
        # table rows
        for score_name, score_dict in sorted(scores.items()) :
            headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
            for i in range(numlabels):
                if score_name in ["F1","Precision","Recall", "G-mean"]:
                    head = "  %-20s  %4.1f ± %4.1f  " + numlabels* "%4.1f ± %4.1f  "
                    vals = [v[i] for v in scores[score_name]["lab"]]
                    headv.append(np.mean(vals)*100)
                    headv.append(np.std(vals)*100)
                else:
                    head = "  %-20s  %4.1f ± %4.1f  " + numlabels * "%-11s  " 
                    headv.append("-")
            print(head % tuple(headv))
        print("―"*len(row))


# Training

In [ ]:
def get_graph_node_stats(vec, nearestDocIDs, y_train, bodyTrainTFIDF):   
    vecdense = vec.toarray()[0]
    docids = nearestDocIDs
    trlabels = np.array(y_train)
    labsum = trlabels[docids].sum()
    
    ivec = []
    labmask = []
    for hitdocid in docids:
        value=bodyTrainTFIDF[hitdocid].toarray()[0]
        intersection = (vecdense>0)*(value>0)
        ivec.append(intersection.sum())
        labmask.append(trlabels[hitdocid])
        
    masked_ivec =  np.array(ivec)*np.array(labmask)   
    masked_ivec_neg =  np.array(ivec)*(-1*(np.array(labmask)-1)) 
    ivec = np.array(ivec)
    masked_ivec = np.array(masked_ivec)
    masked_ivec_neg = np.array(masked_ivec_neg)
    
    newvec = [labsum, (vecdense>0).sum(),ivec.max(), ivec.max(), masked_ivec.max(), masked_ivec.min(), masked_ivec_neg.max(), masked_ivec_neg.min()]
    return newvec

## Swarog Model

In [362]:
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray
from bentoml.io import JSON
from annoy import AnnoyIndex
import re

import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast
import torch
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import pickle5 as pickle
import sqlite3
from tqdm import tqdm
 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)

class SwarogModel:
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, 
                                           truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, 
                                       output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return np.array(allembeds)
    
    def train(self, body, labels):
        embeddings = self.transform(body)
        self.cls = LogisticRegression(max_iter=1000)
        self.cls.fit(embeddings, labels)
        self.train_prob = self.cls.predict_proba(embeddings)
        
    def predict(self, body):
        embeddings = self.transform(body)
        self.test_prob = self.cls.predict_proba(embeddings)
        return  self.cls.predict(embeddings) 

using device: cuda


## Graph Model

In [431]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression

# Download stopwords list
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 
    
def call_it2(instance, name, arg):
    return getattr(instance, name)(arg)

class TrustexModel:
    def __init__(self):
        # Lemmatize the stop words
        self.wnl = WordNetLemmatizer()
        self.ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', "'"]
        
    def tfidf(self,body):
        self.tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
        self.tfidf_vectorizer.fit(body)
        self.bodyTrainTFIDF = self.tfidf_vectorizer.transform(body)
        
    def calc_node_stats(self, item):
        id, topIDs = item
        vec = self.bodyTrainTFIDF[id]
        newvec = get_graph_node_stats(vec, topIDs[1:], self.train_labels, self.bodyTrainTFIDF)
        return newvec
        
    def create_graph(self, body, labels):
        self.nn = NearestNeighbors(n_neighbors=10)
        self.nn.fit(self.bodyTrainTFIDF)
        knn_d,knn_idx = self.nn.kneighbors(self.bodyTrainTFIDF)
        self.graph_knn = []
        self.train_labels = labels
        from tqdm import tqdm
        
        with mp.Pool(processes=mp.cpu_count()) as pool:
            func_call_it = functools.partial(call_it2, self, 'calc_node_stats')
            self.graph_knn = list(tqdm(pool.imap(func_call_it, enumerate(knn_idx),chunksize=64), 
                                total=knn_idx.shape[0]))
        print("avg. nodes sim.=",np.mean([x[2]/(x[1]+0.00001) for x in self.graph_knn]))

    def calc_node_stats_test(self, item):
        id, topIDs = item
        vec = self.bodyTestTFIDF[id]
        newvec = get_graph_node_stats(vec, topIDs, self.train_labels, self.bodyTrainTFIDF)
        return newvec
    
    def graph_transform_test_data(self, body):
        
        self.bodyTestTFIDF = self.tfidf_vectorizer.transform(body) 
        knn_test_d,knn_test_idx = self.nn.kneighbors(self.bodyTestTFIDF)
        
        with mp.Pool(processes=mp.cpu_count()) as pool:
            func_call_it = functools.partial(call_it2, self, 'calc_node_stats_test')
            self.graph_test_knn = list(tqdm(pool.imap(func_call_it, enumerate(knn_test_idx),chunksize=64), 
                                total=knn_test_idx.shape[0]))
        
    def train(self, body, labels):
        print("Building similarity graph")
        self.tfidf(body)
        self.create_graph(body, labels)
          
        self.cls = LogisticRegression(max_iter=10000)
        self.cls.fit(self.graph_knn, labels)
        self.trainX = self.graph_knn

    def predict(self, body):
        print("Create test data")
        self.graph_transform_test_data(body)
        y_pred = self.cls.predict(self.graph_test_knn)
        self.testX = self.graph_test_knn
        return y_pred
    


[nltk_data] Downloading package wordnet to /home/rkozik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rkozik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Semantics  

In [365]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [472]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from subject_verb_object_extract import findSVOs, nlp
# Download stopwords list
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

    
    
def call_it2(instance, name, arg):
    return getattr(instance, name)(arg)

class Semantics:
    def __init__(self):
        # Lemmatize the stop words
        self.wnl = WordNetLemmatizer()
        self.ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', "'"]
        self.token_stop = self.transform_sentence(' '.join(stop_words))
        
    def transform_sentence(self, doc):
        tokens = nlp(doc)
        svos = findSVOs(tokens)
        x = []
        for s in svos:
            x.extend(s)
            
        if len(x) == 0:
            return ["empty"]
        return x

    def tfidf(self,body):
        with mp.Pool(processes=mp.cpu_count()) as pool:
            func_call_it = functools.partial(call_it2, self, 'transform_sentence')
            tok_body = list(tqdm(pool.imap(func_call_it, body,chunksize=64), 
                                total=len(body)))
        vectors = [" ".join(x) for x in tok_body]

        self.tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
        self.tfidf_vectorizer.fit(vectors)
        self.bodyTrainTFIDF = self.tfidf_vectorizer.transform(body)
        
    def calc_node_stats(self, item):
        id, topIDs = item
        vec = self.bodyTrainTFIDF[id]
        newvec = get_graph_node_stats(vec, topIDs[1:], self.train_labels, self.bodyTrainTFIDF)
        return newvec
        
    def create_graph(self, body, labels):
        self.nn = NearestNeighbors(n_neighbors=12)
        self.nn.fit(self.bodyTrainTFIDF)
        knn_d,knn_idx = self.nn.kneighbors(self.bodyTrainTFIDF)
        self.graph_knn = []
        self.train_labels = labels
        from tqdm import tqdm
        
        with mp.Pool(processes=mp.cpu_count()) as pool:
            func_call_it = functools.partial(call_it2, self, 'calc_node_stats')
            self.graph_knn = list(tqdm(pool.imap(func_call_it, enumerate(knn_idx),chunksize=64), 
                                total=knn_idx.shape[0]))
        print("avg. nodes sim.=",np.mean([x[2]/(x[1]+0.00001) for x in self.graph_knn]))

    def calc_node_stats_test(self, item):
        id, topIDs = item
        vec = self.bodyTestTFIDF[id]
        newvec = get_graph_node_stats(vec, topIDs, self.train_labels, self.bodyTrainTFIDF)
        return newvec
    
    def graph_transform_test_data(self, body):
        
        self.bodyTestTFIDF = self.tfidf_vectorizer.transform(body) 
        knn_test_d,knn_test_idx = self.nn.kneighbors(self.bodyTestTFIDF)
        
        with mp.Pool(processes=mp.cpu_count()) as pool:
            func_call_it = functools.partial(call_it2, self, 'calc_node_stats_test')
            self.graph_test_knn = list(tqdm(pool.imap(func_call_it, enumerate(knn_test_idx),chunksize=64), 
                                total=knn_test_idx.shape[0]))     
              
    def train(self, body, labels):
        print("Building similarity graph")
        self.tfidf(body)
        self.create_graph(body, labels)
          
        self.cls = LogisticRegression(max_iter=10000)
        self.cls.fit(self.graph_knn, labels)
        self.trainX = self.graph_knn

    def predict(self, body):
        self.graph_transform_test_data(body)
        y_pred = self.cls.predict(self.graph_test_knn)
        self.testX = self.graph_test_knn
        return y_pred
    


[nltk_data] Downloading package wordnet to /home/rkozik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rkozik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [468]:
bootstrap_size = 500
if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / len(body[train])
    bootstrap = np.random.uniform(size=len(body[train])) < bootstrap_factor
    print(bootstrap[:10])

[False False False False False False False False False  True]


In [473]:
sema = Semantics()
sema.train(body[train][bootstrap],labels[train][bootstrap])

Building similarity graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:00<00:00, 2616.42it/s]


avg. nodes sim.= 0.4060444801421384


In [474]:
ypred = sema.predict(body[test])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 498/498 [00:00<00:00, 2081.06it/s]


In [475]:
met = Metrics()
met.update(labels[test], ypred)
met.print_table()

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              73.3 ±  0.0  -            -            
  Balanced Accuracy     70.8 ±  0.0  -            -            
  F1                    72.7 ±  0.0  63.6 ±  0.0  78.9 ±  0.0  
  G-mean                70.7 ±  0.0  69.6 ±  0.0  69.6 ±  0.0  
  Precision             73.0 ±  0.0  70.7 ±  0.0  74.6 ±  0.0  
  Recall                73.3 ±  0.0  57.7 ±  0.0  83.8 ±  0.0  
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――



## Infrence 

In [367]:
from sklearn.ensemble import RandomForestClassifier

class Inference:
    def train(self,graph, content, semantics, labels):
        newX=[]
        for i,vec in enumerate(content):
            v2 = np.append(content[i], graph[i]) 
            if semantics:
                v2 = np.append(v2, semantics[i])
            newX.append(v2)
            
        self.inf = RandomForestClassifier(max_depth=12)
        self.inf.fit(newX, labels)
        
    def predict(self, graph, content, semantics):
        newTest=[]
        for i,vec in enumerate(content):
            v2 = np.append(content[i], graph[i]) 
            if semantics:
                v2 = np.append(v2, semantics[i])
            newTest.append(v2)
    
        return self.inf.predict(newTest)

# Experiments

In [368]:
loader = DatasetLoader()
body, labels, total_number_of_claims = loader.get("covidfn")
X=range(0,total_number_of_claims)
train, test = list(rskf.split(X, labels))[0]

loading covidfn
total_number_of_claims= 10201
labels fake= 9727 real= 474


## fnkdd

In [476]:
loader = DatasetLoader()

for dataset in ["fnkdd"]:
    body, labels, total_number_of_claims = loader.get(dataset)
    X=range(0,total_number_of_claims)
    
    trustex_quality = Metrics()
    swarog_quality = Metrics()
    semantics_quality = Metrics()
    inf3_quality = Metrics()
    inf2_quality = Metrics()
    

    for fold_idx, (train, test) in enumerate(rskf.split(X, labels)):
        print(f"fold-{fold_idx}")    
        
        bootstrap_size = 0
        if bootstrap_size != 0:
            bootstrap_factor = bootstrap_size / len(train)
            bootstrap = np.random.uniform(size=len(train)) < bootstrap_factor
            train=train[bootstrap]
        
        swarog = SwarogModel()
        trustex = TrustexModel()
        inference2 = Inference()
        inference3 = Inference()
        semantics = Semantics()
        
        print("semantic")
        semantics.train(body[train],labels[train])
        ypred = semantics.predict(body[test])
        semantics_quality.update(labels[test], ypred)
    
        print("trustex")
        trustex.train(body[train],labels[train])
        ypred = trustex.predict(body[test])
        trustex_quality.update(labels[test], ypred)
        
        print("swarog")
        swarog.train(body[train],labels[train])
        ypred = swarog.predict(body[test])
        swarog_quality.update(labels[test], ypred)

        print("inference")
        inference2.train(trustex.graph_knn, swarog.train_prob, None, labels[train])
        newpred = inference2.predict(trustex.graph_test_knn, swarog.test_prob, None)
        inf2_quality.update(labels[test], newpred)

        
        inference3.train(trustex.graph_knn, swarog.train_prob, semantics.trainX, labels[train])
        newpred = inference3.predict(trustex.graph_test_knn, swarog.test_prob, semantics.testX)
        inf3_quality.update(labels[test], newpred)

        

print("Semantic:")
semantics_quality.print_table()

print("Symbolic:")
trustex_quality.print_table()

print("Deep:")
swarog_quality.print_table()

print("Symb+Deep:")
inf2_quality.print_table()

print("ALL:")
inf3_quality.print_table()


loading fnkdd
total_number_of_claims= 4986
labels fake= 2972 real= 2014
fold-0
semantic
Building similarity graph


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4487/4487 [00:02<00:00, 1503.28it/s]


avg. nodes sim.= 0.4704687141506022


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 629.34it/s]


trustex
Building similarity graph


...

avg. nodes sim.= 0.4409309824844639
Create test data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 498/498 [00:00<00:00, 597.29it/s]


swarog


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:02<00:00, 58.32it/s]


inference
Semantic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              73.8 ±  2.2  -            -            
  Balanced Accuracy     70.8 ±  2.4  -            -            
  F1                    73.0 ±  2.4  63.0 ±  3.5  79.8 ±  1.6  
  G-mean                70.8 ±  2.5  69.1 ±  2.8  69.1 ±  2.8  
  Precision             73.8 ±  2.3  73.4 ±  3.1  74.0 ±  1.9  
  Recall                73.8 ±  2.2  55.2 ±  3.9  86.4 ±  1.5  
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Symbolic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              73.4 ±  1.7  -            -            
  Balanced Accuracy     70.3 ±  1.9  -            -            
  F1             

## covidfn

In [442]:
loader = DatasetLoader()

for dataset in ["covidfn"]:
    body, labels, total_number_of_claims = loader.get(dataset)
    X=range(0,total_number_of_claims)
    
    trustex_quality = Metrics()
    swarog_quality = Metrics()
    semantics_quality = Metrics()
    inf3_quality = Metrics()
    inf2_quality = Metrics()
    

    for fold_idx, (train, test) in enumerate(rskf.split(X, labels)):
        print(f"fold-{fold_idx}")    
        
        bootstrap_size = 0
        if bootstrap_size != 0:
            bootstrap_factor = bootstrap_size / len(train)
            bootstrap = np.random.uniform(size=len(train)) < bootstrap_factor
            train=train[bootstrap]
        
        swarog = SwarogModel()
        trustex = TrustexModel()
        inference2 = Inference()
        inference3 = Inference()
        semantics = Semantics()
        
        print("semantic")
        semantics.train(body[train],labels[train])
        ypred = semantics.predict(body[test])
        semantics_quality.update(labels[test], ypred)
    
        print("trustex")
        trustex.train(body[train],labels[train])
        ypred = trustex.predict(body[test])
        trustex_quality.update(labels[test], ypred)
        
        print("swarog")
        swarog.train(body[train],labels[train])
        ypred = swarog.predict(body[test])
        swarog_quality.update(labels[test], ypred)

        print("inference")
        inference2.train(trustex.graph_knn, swarog.train_prob, None, labels[train])
        newpred = inference2.predict(trustex.graph_test_knn, swarog.test_prob, None)
        inf2_quality.update(labels[test], newpred)

        
        inference3.train(trustex.graph_knn, swarog.train_prob, semantics.trainX, labels[train])
        newpred = inference3.predict(trustex.graph_test_knn, swarog.test_prob, semantics.testX)
        inf3_quality.update(labels[test], newpred)

        

print("Semantic:")
semantics_quality.print_table()

print("Symbolic:")
trustex_quality.print_table()

print("Deep:")
swarog_quality.print_table()

print("Symb+Deep:")
inf2_quality.print_table()

print("ALL:")
inf3_quality.print_table()


loading covidfn
total_number_of_claims= 10201
labels fake= 9727 real= 474
fold-0
semantic
Building similarity graph


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 9180/9180 [00:11<00:00, 804.83it/s]
/media/rkozik/02FF-A831/repos/swarog_exp/env.hator/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['because', 'couldn', 'didn', 'doesn', 'down', 'here', 'hers', 'ma', 'mightn', 'most', 'my', 'ourselves', 'should', 'some', 'such', 'themselves', 'those', 'wasn', 'wouldn', 'yourselves'] not in stop_words.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 9180/9180 [00:02<00:00, 4027.81it/s]


avg. nodes sim.= 0.6416482573104035


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1021/1021 [00:01<00:00, 579.56it/s]


trustex
Building similarity graph


...

avg. nodes sim.= 0.6016977018324124
Create test data


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020/1020 [00:01<00:00, 565.60it/s]


swarog


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [00:01<00:00, 200.06it/s]


inference
Semantic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              97.4 ±  0.3  -            -            
  Balanced Accuracy     74.0 ±  2.0  -            -            
  F1                    97.0 ±  0.3  63.4 ±  4.0  98.7 ±  0.1  
  G-mean                70.1 ±  2.7  69.2 ±  2.8  69.2 ±  2.8  
  Precision             97.3 ±  0.4  93.4 ±  6.1  97.5 ±  0.2  
  Recall                97.4 ±  0.3  48.1 ±  4.0  99.8 ±  0.2  
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Symbolic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              97.4 ±  0.4  -            -            
  Balanced Accuracy     73.1 ±  3.5  -            -            
  F1             

## liar

In [443]:
loader = DatasetLoader()

for dataset in ["liar"]:
    body, labels, total_number_of_claims = loader.get(dataset)
    X=range(0,total_number_of_claims)
    
    trustex_quality = Metrics()
    swarog_quality = Metrics()
    semantics_quality = Metrics()
    inf3_quality = Metrics()
    inf2_quality = Metrics()
    

    for fold_idx, (train, test) in enumerate(rskf.split(X, labels)):
        print(f"fold-{fold_idx}")    
        
        bootstrap_size = 0
        if bootstrap_size != 0:
            bootstrap_factor = bootstrap_size / len(train)
            bootstrap = np.random.uniform(size=len(train)) < bootstrap_factor
            train=train[bootstrap]
        
        swarog = SwarogModel()
        trustex = TrustexModel()
        inference2 = Inference()
        inference3 = Inference()
        semantics = Semantics()
        
        print("semantic")
        semantics.train(body[train],labels[train])
        ypred = semantics.predict(body[test])
        semantics_quality.update(labels[test], ypred)
    
        print("trustex")
        trustex.train(body[train],labels[train])
        ypred = trustex.predict(body[test])
        trustex_quality.update(labels[test], ypred)
        
        print("swarog")
        swarog.train(body[train],labels[train])
        ypred = swarog.predict(body[test])
        swarog_quality.update(labels[test], ypred)

        print("inference")
        inference2.train(trustex.graph_knn, swarog.train_prob, None, labels[train])
        newpred = inference2.predict(trustex.graph_test_knn, swarog.test_prob, None)
        inf2_quality.update(labels[test], newpred)

        
        inference3.train(trustex.graph_knn, swarog.train_prob, semantics.trainX, labels[train])
        newpred = inference3.predict(trustex.graph_test_knn, swarog.test_prob, semantics.testX)
        inf3_quality.update(labels[test], newpred)

        

print("Semantic:")
semantics_quality.print_table()

print("Symbolic:")
trustex_quality.print_table()

print("Deep:")
swarog_quality.print_table()

print("Symb+Deep:")
inf2_quality.print_table()

print("ALL:")
inf3_quality.print_table()


loading liar
total_number_of_claims= 8061
labels fake= 3554 real= 4507
fold-0
semantic
Building similarity graph


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7254/7254 [00:09<00:00, 740.35it/s]
/media/rkozik/02FF-A831/repos/swarog_exp/env.hator/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['because', 'couldn', 'didn', 'doesn', 'down', 'here', 'hers', 'ma', 'mightn', 'most', 'my', 'ourselves', 'should', 'some', 'such', 'themselves', 'those', 'wasn', 'wouldn', 'yourselves'] not in stop_words.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7254/7254 [00:02<00:00, 3364.88it/s]


avg. nodes sim.= 0.435177912552006


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 807/807 [00:01<00:00, 673.26it/s]


trustex
Building similarity graph


...

avg. nodes sim.= 0.37203080815053874
Create test data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 806/806 [00:01<00:00, 699.27it/s]


swarog


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [00:01<00:00, 198.02it/s]


inference
Semantic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              63.3 ±  1.6  -            -            
  Balanced Accuracy     62.0 ±  1.6  -            -            
  F1                    62.9 ±  1.6  68.9 ±  1.4  55.2 ±  2.0  
  G-mean                62.0 ±  1.6  61.1 ±  1.7  61.1 ±  1.7  
  Precision             63.0 ±  1.6  65.4 ±  1.2  59.8 ±  2.2  
  Recall                63.3 ±  1.6  72.8 ±  1.8  51.2 ±  2.0  
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Symbolic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              62.6 ±  0.8  -            -            
  Balanced Accuracy     61.4 ±  0.7  -            -            
  F1             

## mmcovid

In [444]:
loader = DatasetLoader()

for dataset in ["mmcovid"]:
    body, labels, total_number_of_claims = loader.get(dataset)
    X=range(0,total_number_of_claims)
    
    trustex_quality = Metrics()
    swarog_quality = Metrics()
    semantics_quality = Metrics()
    inf3_quality = Metrics()
    inf2_quality = Metrics()
    

    for fold_idx, (train, test) in enumerate(rskf.split(X, labels)):
        print(f"fold-{fold_idx}")    
        
        bootstrap_size = 0
        if bootstrap_size != 0:
            bootstrap_factor = bootstrap_size / len(train)
            bootstrap = np.random.uniform(size=len(train)) < bootstrap_factor
            train=train[bootstrap]
        
        swarog = SwarogModel()
        trustex = TrustexModel()
        inference2 = Inference()
        inference3 = Inference()
        semantics = Semantics()
        
        print("semantic")
        semantics.train(body[train],labels[train])
        ypred = semantics.predict(body[test])
        semantics_quality.update(labels[test], ypred)
    
        print("trustex")
        trustex.train(body[train],labels[train])
        ypred = trustex.predict(body[test])
        trustex_quality.update(labels[test], ypred)
        
        print("swarog")
        swarog.train(body[train],labels[train])
        ypred = swarog.predict(body[test])
        swarog_quality.update(labels[test], ypred)

        print("inference")
        inference2.train(trustex.graph_knn, swarog.train_prob, None, labels[train])
        newpred = inference2.predict(trustex.graph_test_knn, swarog.test_prob, None)
        inf2_quality.update(labels[test], newpred)

        
        inference3.train(trustex.graph_knn, swarog.train_prob, semantics.trainX, labels[train])
        newpred = inference3.predict(trustex.graph_test_knn, swarog.test_prob, semantics.testX)
        inf3_quality.update(labels[test], newpred)

        

print("Semantic:")
semantics_quality.print_table()

print("Symbolic:")
trustex_quality.print_table()

print("Deep:")
swarog_quality.print_table()

print("Symb+Deep:")
inf2_quality.print_table()

print("ALL:")
inf3_quality.print_table()


loading mmcovid
total_number_of_claims= 7332
labels fake= 2028 real= 5304
fold-0
semantic
Building similarity graph


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 6598/6598 [00:38<00:00, 172.57it/s]
/media/rkozik/02FF-A831/repos/swarog_exp/env.hator/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['because', 'couldn', 'didn', 'doesn', 'down', 'here', 'hers', 'ma', 'mightn', 'most', 'my', 'ourselves', 'should', 'some', 'such', 'themselves', 'those', 'wasn', 'wouldn', 'yourselves'] not in stop_words.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 6598/6598 [00:03<00:00, 2192.60it/s]


avg. nodes sim.= 0.32675068766867355


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 734/734 [00:01<00:00, 641.51it/s]


trustex
Building similarity graph


...

avg. nodes sim.= 0.55790444409433
Create test data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 733/733 [00:01<00:00, 596.57it/s]


swarog


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184/184 [00:01<00:00, 121.06it/s]


inference
Semantic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              81.6 ±  1.0  -            -            
  Balanced Accuracy     68.8 ±  1.6  -            -            
  F1                    79.1 ±  1.3  88.4 ±  0.6  54.6 ±  3.2  
  G-mean                67.6 ±  1.8  62.5 ±  2.5  62.5 ±  2.5  
  Precision             82.3 ±  1.2  81.0 ±  0.8  85.7 ±  2.7  
  Recall                81.6 ±  1.0  97.4 ±  0.5  40.1 ±  3.1  
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Symbolic:
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Score                 Average      Lab:1        Lab:2       
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  Accuracy              90.8 ±  1.0  -            -            
  Balanced Accuracy     86.5 ±  1.4  -            -            
  F1             